# Garbage Classification — Full notebook with image preprocessing (step-by-step)

This notebook:
- Loads the garbage classification dataset (from folder structure),
- Applies an **image preprocessing pipeline** (histogram equalization, blur, Sobel edges) using a `preprocessing_function` for `ImageDataGenerator`,
- Trains two models (DenseNet121 and ResNet101V2) with transfer learning,
- Evaluates and visualizes results (confusion matrix, classification report),
- Includes visualization of **original vs enhanced** images.

**Note:** Adjust `path` if your dataset location is different (example: Google Drive or Kaggle dataset).


In [ ]:
# 1) Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image

import tensorflow as tf
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Dense , Flatten , Dropout , GlobalAveragePooling2D
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.applications import DenseNet121, ResNet101V2
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# optimizer
from tensorflow.keras.optimizers import Adam, AdamW

import cv2

import warnings
warnings.filterwarnings('ignore')

print('TensorFlow version:', tf.__version__)


In [ ]:
# 2) Parameters & dataset path
# Change this path if your dataset is elsewhere (e.g., Google Drive or local)
path = '/kaggle/input/garbage-classification/garbage_classification'  # <-- keep or update
img_size = 128
batch_size = 32
random_state = 42


In [ ]:
# 3) Build dataframe of image filepaths and labels
filepaths = []
labels = []

for root, dirs, files in os.walk(path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            filepath = os.path.join(root, file)
            filepaths.append(filepath)
            label = os.path.basename(root)
            labels.append(label)

data_df = pd.DataFrame({'filepath': filepaths, 'original_label': labels})
print('Total images found:', len(data_df))
data_df.head()


In [ ]:
# 4) Unify labels (example: unify various glass subfolders into 'glass')
def unify_glass_labels(label):
    if 'glass' in label.lower():
        return 'glass'
    return label

data_df['unified_label'] = data_df['original_label'].apply(unify_glass_labels)
data_df.drop(columns=['original_label'], inplace=True)

print('Class distribution:')
display(data_df['unified_label'].value_counts())


In [ ]:
# 5) Train / Test split (stratified)
train_df, test_df = train_test_split(
    data_df,
    test_size=0.2,
    stratify=data_df['unified_label'],
    random_state=random_state
)

print('Train samples:', len(train_df))
print('Test samples :', len(test_df))


In [ ]:
# 6) Preprocessing function
# This function is compatible with ImageDataGenerator.preprocessing_function.
# Keras calls the preprocessing_function after rescale (so input here will be float32 in [0,1]).
# We convert back to 0-255 before applying cv2 operations, then return a float array in [0,1].

def enhance_preprocessing(img):
    import numpy as np
    import cv2
    # img: float32 in [0,1], shape (H, W, 3), color order: RGB
    # Convert to uint8 [0,255]
    arr = (img * 255).astype('uint8')
    # Convert RGB -> Grayscale
    gray = cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)
    # Histogram equalization
    equalized = cv2.equalizeHist(gray)
    # Gaussian blur
    blurred = cv2.GaussianBlur(equalized, (3, 3), 0)
    # Sobel edge detection
    sobelx = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    sobel = np.sqrt(sobelx**2 + sobely**2)
    sobel = np.clip(sobel, 0, 255).astype('uint8')
    # Stack back to 3 channels (RGB-like)
    final = np.stack([sobel, sobel, sobel], axis=-1)
    # Convert to float [0,1]
    final = final.astype('float32') / 255.0
    return final


In [ ]:
# 7) Create ImageDataGenerators (with augmentation for training)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=enhance_preprocessing  # apply our enhancement
)

test_gen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=enhance_preprocessing  # apply same preprocessing for evaluation
)

train_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='filepath',
    y_col='unified_label',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True
)

test_data = test_gen.flow_from_dataframe(
    test_df,
    x_col='filepath',
    y_col='unified_label',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False
)


In [ ]:
# 8) Class indices & labels
class_indices = train_data.class_indices
print('Class indices (label -> index):')
print(class_indices)

# Build index -> label mapping for predictions later
index_to_label = {v: k for k, v in class_indices.items()}
classes = [index_to_label[i] for i in range(len(index_to_label))]
print('\nClasses (in model index order):', classes)


In [ ]:
# 9) Visualize: Original vs Enhanced
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Pick a sample image from test_df
sample_fp = test_df['filepath'].iloc[0]
print('Sample filepath:', sample_fp)

orig = img_to_array(load_img(sample_fp, target_size=(img_size, img_size))) / 255.0
enh = enhance_preprocessing(orig)

fig, axes = plt.subplots(1,2, figsize=(10,5))
axes[0].imshow(orig)
axes[0].set_title('Original (rescaled)')
axes[0].axis('off')

axes[1].imshow(enh)
axes[1].set_title('Enhanced (preprocessing_function)')
axes[1].axis('off')
plt.show()


In [ ]:
# 10) Train DenseNet121 (transfer learning)
num_classes = len(class_indices)

base_model = DenseNet121(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')
base_model.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model_DenseNet121 = Model(inputs=base_model.input, outputs=predictions)

optimizer = AdamW(learning_rate=1e-4)
model_DenseNet121.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model_DenseNet121.summary()


In [ ]:
# 11) Fit DenseNet121
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

earlystop = EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy')
checkpoint_path = 'dense121_best.h5'
mc = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1)

epochs = 20

history_dense = model_DenseNet121.fit(
    train_data,
    validation_data=test_data,
    epochs=epochs,
    callbacks=[earlystop, mc]
)


In [ ]:
# 12) Evaluate DenseNet121
loss, accuracy = model_DenseNet121.evaluate(test_data)
print(f'DenseNet121 -> Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

# Predictions
preds = model_DenseNet121.predict(test_data, verbose=1)
y_pred_idx = np.argmax(preds, axis=1)
y_pred_labels = [index_to_label[i] for i in y_pred_idx]
y_true_labels = test_df['unified_label'].values

# Confusion matrix
plt.figure(figsize=(10,8))
cm = confusion_matrix(y_true_labels, y_pred_labels, labels=classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.title('DenseNet121 - Confusion Matrix')
plt.show()

print('\nClassification Report:')
print(classification_report(y_true_labels, y_pred_labels, target_names=classes))


In [ ]:
# 13) Train ResNet101V2 (transfer learning)
base_model = ResNet101V2(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')
base_model.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model_ResNet101V2 = Model(inputs=base_model.input, outputs=predictions)

optimizer = AdamW(learning_rate=1e-4)
model_ResNet101V2.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model_ResNet101V2.summary()


In [ ]:
# 14) Fit ResNet101V2
checkpoint_path_r = 'resnet101v2_best.h5'
mc_r = ModelCheckpoint(checkpoint_path_r, monitor='val_accuracy', save_best_only=True, verbose=1)
earlystop_r = EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy')

history_resnet = model_ResNet101V2.fit(
    train_data,
    validation_data=test_data,
    epochs=epochs,
    callbacks=[earlystop_r, mc_r]
)


In [ ]:
# 15) Evaluate ResNet101V2
loss_r, accuracy_r = model_ResNet101V2.evaluate(test_data)
print(f'ResNet101V2 -> Loss: {loss_r:.4f}, Accuracy: {accuracy_r:.4f}')

# Predictions
preds_r = model_ResNet101V2.predict(test_data, verbose=1)
y_pred_idx_r = np.argmax(preds_r, axis=1)
y_pred_labels_r = [index_to_label[i] for i in y_pred_idx_r]

plt.figure(figsize=(10,8))
cm_r = confusion_matrix(y_true_labels, y_pred_labels_r, labels=classes)
sns.heatmap(cm_r, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.title('ResNet101V2 - Confusion Matrix')
plt.show()

print('\nClassification Report (ResNet101V2):')
print(classification_report(y_true_labels, y_pred_labels_r, target_names=classes))


In [ ]:
# 16) Plot training history (DenseNet121 vs validation)
def plot_history(h, title='Model'):
    plt.figure(figsize=(8,4))
    plt.plot(h.history['accuracy'], label='train_acc')
    plt.plot(h.history['val_accuracy'], label='val_acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title(title)
    plt.show()

plot_history(history_dense, title='DenseNet121 Accuracy')
plot_history(history_resnet, title='ResNet101V2 Accuracy')
